In [1]:
!pip install mtcnn keras_vggface keras_applications

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
! pip install git+https://github.com/rcmalli/keras-vggface.git
!pip install keras_applications --no-deps
filename = "/usr/local/lib/python3.9/dist-packages/keras_vggface/models.py"
text = open(filename).read()
open(filename, "w+").write(text.replace('keras.engine.topology', 'tensorflow.keras.utils'))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-1o8izlhg
  Running command git clone --filter=blob:none --quiet https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-1o8izlhg
  Resolved https://github.com/rcmalli/keras-vggface.git to commit 719ad75625fe51436b9a9479114b3d136847e11a
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


20951

In [4]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from numpy import asarray
from mtcnn.mtcnn import MTCNN
from scipy.spatial.distance import cosine
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
import sys

In [5]:
# extract a single face from an uploaded photograph
detector = MTCNN()
def extract_face(pixels,detector=detector,required_size=(224, 224)):
	# pixels = plt.imread(filename)
	# print(pixels)
	# print(pixels.shape)
	# create the detector, using default weights
	# detect faces in the image
	results = detector.detect_faces(pixels)
	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)
	return face_array
    # resize pixels to the model size

In [6]:
import os

In [7]:
modelPath = "drive/MyDrive/missing child model/vgg_transfer_learning_model.h5"

In [8]:
from tensorflow import keras
model = keras.models.load_model(modelPath)

In [9]:
#the resnet model represents the face in a 2048-dimension feature space.

def get_embedding(filename,model=model):
    # extract faces
    face = extract_face(filename)
    # convert into an array of samples
    sample = [asarray(face, 'float32')]
    # prepare the face for the model, e.g. center pixels
    sample = preprocess_input(sample, version=2)
    # perform prediction
    yhat = model.predict(sample)
    return yhat

In [26]:
def is_match(ID_embedding, subject_embedding):
    # calculate distance between embeddings
    score = cosine(ID_embedding, subject_embedding)
    is_matched = score <= 0.4
    result_dic = {"is_matched": is_matched, "score": score}
    return result_dic

In [11]:
import urllib
def url_to_image(url):
	# download the image, convert it to a NumPy array, and then read
	# it into OpenCV format
	resp = urllib.request.urlopen(url)
	image = np.asarray(bytearray(resp.read()), dtype="uint8")
	image = cv2.imdecode(image, cv2.IMREAD_COLOR)
	# return the image
	return image

In [12]:
!pip install pyngrok==4.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
!ngrok authtoken 2OKjhLzk6P8xGgO11Iz0qruEb5J_5wEJ2ViX6EtUujueYjHdd

from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)   

@app.route('/', methods=['POST'])
def faceClassification():
  try:
    if request.method == 'POST':
        data = request.get_json()
        imgUrl1 = data['image1']
        imgUrl2 = data['image2']

        if imgUrl1 is None or imgUrl2 is None:
            return jsonify({"error": "Invalid arguments"}), 400
        
        print(imgUrl1, imgUrl2, "out")
        img1 = url_to_image(imgUrl1)
        img2 = url_to_image(imgUrl2)
        
        img1_embedding = get_embedding(img1)
        img2_embedding = get_embedding(img2)

        matchResult = is_match(img1_embedding[0], img2_embedding[0])
        return jsonify(matchResult)

  except: 
    return jsonify({"error": "Internal server error occured."}), 500
      
@app.route('/')
def home():
  return {"mess": "hello"}

app.run()

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://e31b-104-154-46-228.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
https://firebasestorage.googleapis.com/v0/b/orphaned-c5d6f.appspot.com/o/case_images%2F1680618687.jpg?alt=media&token=1bb429d0-9065-4b22-ae75-1e8a8fcb0882 https://firebasestorage.googleapis.com/v0/b/orphaned-c5d6f.appspot.com/o/case_images%2F1680618687.jpg?alt=media&token=1bb429d0-9065-4b22-ae75-1e8a8fcb0882 out
1/1 [==============================] - 0s 21ms/step


INFO:werkzeug:127.0.0.1 - - [12/Apr/2023 17:17:39] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2023 17:17:52] "POST / HTTP/1.1" 500 -
